## EGU Topic Modeling and prediction

This notebook uses topic modeling to analyze the EGU conference using the abstracts submitted for the years 2011 to 2018. The notebook can be used to detect trends and visualize topics along these years using one or many categories. The abstracts were parsed into text and ingested into a Apache Solr instance.

We parsed the PDFs using PDFMiner's utility pdf2txt

```sh
ls *.pdf | xargs -n1 -P8 bash -c 'pdf2txt.py -o output/$0.txt -t text $0'
```

**Solr Schema**
 
 
 
```json
doc = {
"entities":[
    "Jeffrey Obelcz  and Warren T. Wood",
    "NRC Postdoctoral Fellow",
    "Naval Research Lab",
    "Seaﬂoor Sciences",
    "United States jbobelcz@gmail.com",
    "Naval",
    "Research Lab",
    "Seaﬂoor Sciences",
    "United States"],
"id": "EGU2018-9778",
"sessions": ["ESSI4.3"],
"file": ["EGU2018-9778"],
"presentation": ["Posters"],
"year": [2018],
"title": ["Towards a Quantitative Understanding of Parameters Driving Submarine Slope Failure: A Machine Learning Approach"],
"category": ["ESSI"],
"abstract":["Submarine slope failure is a ubiquitous process and dominant pathway for sediment and organic carbon ﬂux from continental margins to the deep sea. Slope failure occurs over a wide range of temporal and spatial scales ..."]
}

```

## Application

This notebook can be used to analyze what a corpora of scientific text talks about, in this case we used EGU abstracts but it can be used on any text corpora.

The topics are examined using LDAVis, which displays the topics on an X-Y plot (intertopic distance map). Topic are represented by circles whose areas are proportional to the relative prevalences of the topics in the corpus. In the display the user can enter a topic number (note 1-base numbering vs. 0-base numbering in Topic List below cells 5 and 7); the terms are displayed on the right, ranked by significance (weight). A topic can be selected on the fly by hovering over its circle; clicking selects that topic. A user can also click on a term in the RH panel to show the topics in which that term occurs. 

The slider at the top of the RH panel allows the user to vary the “saliency”, i.e. uniqueness of the terms to that topic. A value of 0.6 is optimal, according to the authors of the algorithm. Blue bars represent overall term frequency while red bars show term frequency within the topic, which will be different when the saliency Is selected be less than one.

Please see the annotated image of the LDAVis display.

![](https://raw.githubusercontent.com/USCDataScience/pdi-topics/master/notebooks/pdi/img/pylda.png)



The final cell of this notebook lists the titles and IDs of abstracts belonging to a specified topic.

* category: the Program Group, e.g. CL, AS etc. Keep in mind that these codes have changed through the years.
* presentation: Presentation type: oral, poster, pico etc.
* session: the session that the abstract appeared in; sessions may be co-organized.

-----
EGU Programme Groups:

### Disciplinary Sessions 

Geodynamics (GD) -
Geosciences Instrumentation & Data Systems (GI) -
Geomorphology (GM) -
Geochemistry, Mineralogy, Petrology & Volcanology (GMPV) -
Hydrological Sciences (HS) -
Natural Hazards (NH) -
Nonlinear Processes in Geosciences (NP) -
Ocean Sciences (OS) -
Planetary & Solar System Sciences (PS) -
Seismology (SM) -
Stratigraphy, Sedimentology & Palaeontology (SSP) -
Soil System Sciences (SSS) -
Solar-Terrestrial Sciences (ST) -
Tectonics & Structural Geology (TS) -
Atmospheric Sciences (AS) -
Biogeosciences (BG) -
Climate: Past, Present, Future (CL) -
Cryospheric Sciences (CR) -
Earth Magnetism & Rock Physics (EMRP) -
Energy, Resources and the Environment (ERE) -
Earth & Space Science Informatics (ESSI) -
Geodesy (G) -
Geodynamics (GD) -
Geosciences Instrumentation & Data Systems (GI) -
Geomorphology (GM) -
Geochemistry, Mineralogy, Petrology & Volcanology (GMPV) -
Hydrological Sciences (HS) -
Natural Hazards (NH) -
Nonlinear Processes in Geosciences (NP) -
Ocean Sciences (OS) -
Planetary & Solar System Sciences (PS) -
Seismology (SM) -
Stratigraphy, Sedimentology & Palaeontology (SSP) -
Soil System Sciences (SSS) -
Solar-Terrestrial Sciences (ST) -
Tectonics & Structural Geology (TS) -

### Union Sessions

Union Symposia (US)
Great Debates (GDB)
Medal Lectures (ML)
Short courses (SC)
Educational and Outreach Symposia (EOS)
EGU Plenary, Ceremonies and Networking (PCN)
Feedback and administrative meetings (FAM)
Townhall and splinter meetings (TSM)
Side events (SEV)
Press conferences (PC)

#### Interdisciplinary Events (IE)


In [ ]:
# Cell 1: Import requirements
import urllib
import json
import string
import pandas as pd
import random

pseudo_rand = [random.choice(string.ascii_letters) for i in range(4)]
seed = ''.join(pseudo_rand)

# NLP libraries
import spacy
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# We are using spacy as a parser so we disable their other capabilities to speed up things
nlp = spacy.load('en', disable=['tagger', 'ner'])


In [ ]:
# Cell 2: Querying Solr

# terms = ['ice', 'climate'] to include only abstracts with specified terms
terms = ['*']
years = ['2011', '2018']
entities = ['*']
sessions = ['NH']
top_documents = 1000

cursorMark = '*'

solr_documents = []
solr_root = 'http://integration.pdi-solr.labs.nsidc.org/solr/egu/select?indent=on&'
more_results = True


if terms[0] != '*':
    terms_wirldcard = ['*' + t + '*' for t in terms]
else:
    terms_wirldcard = ['*']
    
if sessions[0] != '*':
    sessions_wirldcard = ['*' + s + '*' for s in sessions]
else:
    sessions_wirldcard = ['*']
    
if entities[0] != '*':
    entities_wirldcard = ['*' + e + '*' for e in entities]
else:
    entities_wirldcard = ['*']

# Return "page_size" documents with each Solr query until complete
page_size = 1000
current_round = 1
terms_query = '%20OR%20abstract:'.join(terms_wirldcard)
years_query = '%20OR%20year:'.join(years)  
entities_query = '%20OR%20entities:'.join(entities_wirldcard)
sessions_query = '%20OR%20sessions:'.join(sessions_wirldcard)
query_string = 'q=(abstract:{}%20AND%20abstract:/.{{2}}.*/%20AND%20NOT%20title:/.{{300}}.*/)%20AND%20(year:{})' + \
                '%20AND%20(entities:{})%20AND%20(sessions:{})&wt=json&rows={}&cursorMark={}&sort=random_*{}%20desc,id%20desc'
while (more_results):
    solr_query = query_string.format(terms_query,
                                     years_query,
                                     entities_query,
                                     sessions_query,
                                     page_size,
                                     cursorMark,
                                     seed)
    solr_url = solr_root + solr_query
    print('Querying: \n' + solr_url)
    current_round += 1
    req = urllib.request.Request(solr_url)
    # parsing response
    r = urllib.request.urlopen(req).read()
    json_response = json.loads(r.decode('utf-8'))
    solr_documents.extend(json_response['response']['docs'])
    nextCursorMark = json_response['nextCursorMark']
    if (nextCursorMark == cursorMark) or top_documents <= (current_round * page_size):
        more_results = False
        break
    else: 
        cursorMark = nextCursorMark

total_found = json_response['response']['numFound']
print("Processing {0} out of {1} total. \n".format(len(solr_documents), total_found))

In [ ]:
# Cell 3: Cleaning our documents 

import string

# Helper function
def flatten(top_list):
    for inner in top_list:
        if isinstance(inner, (list,tuple)):
            for j in flatten(inner):
                yield j
        else:
            yield inner


stop_years = {str(year) for year in range(2000,2020)}

nlp.Defaults.stop_words |= stop_years
nlp.Defaults.stop_words |= ENGLISH_STOP_WORDS

nlp.Defaults.stop_words |= {
                            'area', 'data', 'event', 'use', 'group', 'research', 'model',
                            'metadata', 'content', 'sharing', 'previous', 'http','study',
                            'datum', 'result', 'different', 'change', 'et_al','change', 'different'
                           }

# After we generate the bigrams there are terms that we might want to remove including bigrams.
post_process_stopwords = {'sea_ice', 'water'}

ALL_STOP_WORDS = ENGLISH_STOP_WORDS.union(nlp.Defaults.stop_words)


def remove_stop_words(text):
    words = text.split()
    cleaned_text = [w for w in words if w not in ALL_STOP_WORDS]
    cleaned_text = ' '.join([w for w in cleaned_text if len(w) > 2])
    return cleaned_text

document_list = []
# bigram corpus will contain an array of documents and their tokens, with bigram tokens included
bigram_corpus = []

for doc in solr_documents:
    bigrams = remove_stop_words(doc['bigrams'][0])
    if 'sessions' in doc:
        sessions = doc['sessions'][0]
    else:
        sessions = 'NAN'
    if 'category' in doc:
        category = doc['category'][0]
    else:
        category = 'NAN'
    document_list.append({ 'id': doc['id'],
                                   'text': doc['abstract'][0],
                                   'bigrams': bigrams,
                                   'year': str(doc['year'][0]),
                                   'title': doc['title'][0],
                                   'category': category.replace('<',''),
                                   'sessions':sessions})
    bigram_corpus.append(bigrams)

df = pd.DataFrame.from_dict(document_list)
axis_category = pd.DataFrame(df.groupby(['category', 'year'])['category'].count()).rename(columns={'category': 'count'})
print(axis_category)
# print(len(df.iloc[0]['bigrams']))
# print(solr_documents[0]['bigrams'][0])


### ScatterText using bigrams as input

In [ ]:
# Cell 5, using ScatterText


import scattertext as st

# scattertext categories (year or session)
scatter_category = 'year'

if scatter_category == 'year':
    comparing = [years[0],years[1]]
else:
    comparing = [sessions[0],sessions[1]]
    scatter_category = 'category'

    
# We create a corpus using Scatter's built-in method.
scatter_corpus = st.CorpusFromPandas(df, 
                             category_col=scatter_category, 
                             text_col='bigrams',
                             nlp=nlp).build()

html = st.produce_scattertext_explorer(scatter_corpus,
          category=comparing[0],
          category_name=comparing[0],
          not_category_name=comparing[1],
          metadata=scatter_corpus.get_df()['title'],
          minimum_term_frequency=5,
          width_in_pixels=700)

open("scattertext.html", 'wb').write(html.encode('utf-8'))
from IPython.display import IFrame
from IPython.core.display import display, HTML
print ("Loading plot...")
display(IFrame(src='scattertext.html', width=900, height=800))
# The search box is not working, presumably because Jupyter getting in the way of scattertext js libraries.

In [ ]:
# Cell 6: Listing papers containing a particular ngram

terms = set(['tsunami_wave','sea_ice'])

top_n = 10

def createLink(doc):
    baseURL = 'https://meetingorganizer.copernicus.org/EGU' + str(doc['year']) + '/' + doc['id'] + '.pdf'
    return baseURL

# Ode to Python's comprehension lists
matches  = [doc for doc in document_list if set(doc['bigrams'].split()).intersection(terms)]

from IPython.core.display import display, HTML
# Let's predict the first 10 documents
for doc in matches[0:top_n]:
    display(HTML('<br>Abstract <a href="{}" target="_blank">{}</a> '.format(
        createLink(doc),
        doc['id']))) 
